In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6232,2020-12-16T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1188,183,1371,33805,...,2059,NaN,NaN,81256,1108575,744403.0,NaN,18.0,NaN,NaN
6233,2020-12-16T17:00:00,ITA,9,Toscana,43.769231,11.255889,1101,202,1303,13732,...,3285,NaN,NaN,113610,1746053,1014930.0,NaN,12.0,Positivi diagnosticati solo con test antigenic...,NaN
6234,2020-12-16T17:00:00,ITA,10,Umbria,43.106758,12.388247,270,47,317,4006,...,541,NaN,NaN,26587,463909,239527.0,NaN,4.0,NaN,NaN
6235,2020-12-16T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,67,7,74,449,...,360,NaN,NaN,6943,59111,35825.0,NaN,1.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6232,2020-12-16T17:00:00,19,Sicilia,1188,183,1371,33805,35176,-793,1065,44021,2059,NaN,NaN,81256,1108575,744403.0,18.0,NaN,NaN
6233,2020-12-16T17:00:00,9,Toscana,1101,202,1303,13732,15035,-1229,489,95290,3285,NaN,NaN,113610,1746053,1014930.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
6234,2020-12-16T17:00:00,10,Umbria,270,47,317,4006,4323,-237,169,21723,541,NaN,NaN,26587,463909,239527.0,4.0,NaN,NaN
6235,2020-12-16T17:00:00,2,Valle d'Aosta,67,7,74,449,523,-4,38,6060,360,NaN,NaN,6943,59111,35825.0,1.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-16', '2020-12-15')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-16T17:00:00,1,Piemonte,3698,257,3955,44148,48103,-2867,1215,132982,7302,NaN,NaN,188387,1801939,1054003.0,8.0,NaN,NaN
1,2020-12-16T17:00:00,2,Valle d'Aosta,67,7,74,449,523,-4,38,6060,360,NaN,NaN,6943,59111,35825.0,1.0,NaN,NaN
2,2020-12-16T17:00:00,3,Lombardia,4946,629,5575,64158,69733,-6157,2994,356175,24097,NaN,NaN,450005,4514914,2489727.0,18.0,NaN,NaN
3,2020-12-16T17:00:00,5,Veneto,2698,347,3045,91180,94225,1535,3817,101313,5069,NaN,NaN,200607,3043792,1140117.0,31.0,NaN,NaN
4,2020-12-16T17:00:00,6,Friuli Venezia Giulia,634,54,688,13063,13751,-188,779,27009,1339,NaN,NaN,42099,837745,317196.0,7.0,NaN,NaN
5,2020-12-16T17:00:00,7,Liguria,755,71,826,7203,8029,-86,275,45584,2705,NaN,NaN,56318,660289,308177.0,5.0,Dei 120854 test antigenici rapidi sono riferit...,NaN
6,2020-12-16T17:00:00,8,Emilia-Romagna,2773,212,2985,59861,62846,-1788,1238,79151,6797,NaN,NaN,148794,2367376,1162546.0,9.0,Il dato odierno cumulativo dei tamponi antigen...,NaN
7,2020-12-16T17:00:00,9,Toscana,1101,202,1303,13732,15035,-1229,489,95290,3285,NaN,NaN,113610,1746053,1014930.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-16T17:00:00,10,Umbria,270,47,317,4006,4323,-237,169,21723,541,NaN,NaN,26587,463909,239527.0,4.0,NaN,NaN
9,2020-12-16T17:00:00,11,Marche,448,75,523,10608,11131,-759,480,23112,1437,NaN,NaN,35680,467740,273780.0,2.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-15T17:00:00,1,Piemonte,3761,266,4027,46943,50970,-2366,1106,128989,7213,NaN,NaN,187172,1784446,1046569.0,5.0,NaN,NaN
1,2020-12-15T17:00:00,2,Valle d'Aosta,74,6,80,447,527,-61,20,6022,356,NaN,NaN,6905,58718,35559.0,0.0,NaN,NaN
2,2020-12-15T17:00:00,3,Lombardia,4996,656,5652,70238,75890,-2431,2404,347130,23991,NaN,NaN,447011,4477309,2478151.0,28.0,NaN,NaN
3,2020-12-15T17:00:00,5,Veneto,2694,346,3040,89650,92690,162,3320,99108,4992,NaN,NaN,196790,3023689,1134294.0,45.0,NaN,NaN
4,2020-12-15T17:00:00,6,Friuli Venezia Giulia,654,56,710,13229,13939,-149,829,26082,1299,NaN,NaN,41320,828919,315194.0,3.0,NaN,NaN
5,2020-12-15T17:00:00,7,Liguria,768,71,839,7276,8115,-165,202,45234,2694,NaN,NaN,56043,655617,307046.0,3.0,NaN,NaN
6,2020-12-15T17:00:00,8,Emilia-Romagna,2797,214,3011,61623,64634,-352,1238,76205,6719,NaN,NaN,147558,2349809,1157563.0,17.0,NaN,NaN
7,2020-12-15T17:00:00,9,Toscana,1156,214,1370,14894,16264,-1215,332,93619,3238,NaN,NaN,113121,1733863,1011122.0,15.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-15T17:00:00,10,Umbria,288,46,334,4226,4560,-175,179,21323,535,NaN,NaN,26418,460748,238473.0,7.0,NaN,NaN
9,2020-12-15T17:00:00,11,Marche,467,78,545,11345,11890,-912,228,21882,1428,NaN,NaN,35200,460780,269841.0,3.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-16T17:00:00,1,Piemonte,3698,257,3955,44148,48103,-2867,1215,132982,7302,NaN,NaN,188387,1801939,1054003.0,8.0,NaN,NaN
1,2020-12-16T17:00:00,2,Valle d'Aosta,67,7,74,449,523,-4,38,6060,360,NaN,NaN,6943,59111,35825.0,1.0,NaN,NaN
2,2020-12-16T17:00:00,3,Lombardia,4946,629,5575,64158,69733,-6157,2994,356175,24097,NaN,NaN,450005,4514914,2489727.0,18.0,NaN,NaN
3,2020-12-16T17:00:00,4,Trentino-Alto Adige,617,72,689,12767,13456,119,529,31383,1476,0.0,0.0,46315,757305,302339.0,4.0,0,NaN
4,2020-12-16T17:00:00,5,Veneto,2698,347,3045,91180,94225,1535,3817,101313,5069,NaN,NaN,200607,3043792,1140117.0,31.0,NaN,NaN
5,2020-12-16T17:00:00,6,Friuli Venezia Giulia,634,54,688,13063,13751,-188,779,27009,1339,NaN,NaN,42099,837745,317196.0,7.0,NaN,NaN
6,2020-12-16T17:00:00,7,Liguria,755,71,826,7203,8029,-86,275,45584,2705,NaN,NaN,56318,660289,308177.0,5.0,Dei 120854 test antigenici rapidi sono riferit...,NaN
7,2020-12-16T17:00:00,8,Emilia-Romagna,2773,212,2985,59861,62846,-1788,1238,79151,6797,NaN,NaN,148794,2367376,1162546.0,9.0,Il dato odierno cumulativo dei tamponi antigen...,NaN
8,2020-12-16T17:00:00,9,Toscana,1101,202,1303,13732,15035,-1229,489,95290,3285,NaN,NaN,113610,1746053,1014930.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-16T17:00:00,10,Umbria,270,47,317,4006,4323,-237,169,21723,541,NaN,NaN,26587,463909,239527.0,4.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-15T17:00:00,1,Piemonte,3761,266,4027,46943,50970,-2366,...,128989,7213,NaN,NaN,187172,1784446,1046569.0,5.0,NaN,NaN
1,1,2020-12-15T17:00:00,2,Valle d'Aosta,74,6,80,447,527,-61,...,6022,356,NaN,NaN,6905,58718,35559.0,0.0,NaN,NaN
2,2,2020-12-15T17:00:00,3,Lombardia,4996,656,5652,70238,75890,-2431,...,347130,23991,NaN,NaN,447011,4477309,2478151.0,28.0,NaN,NaN
3,19,2020-12-15T17:00:00,4,Trentino-Alto Adige,620,77,697,12640,13337,171,...,30997,1452,0.0,0.0,45786,751936,301384.0,3.0,0,NaN
4,3,2020-12-15T17:00:00,5,Veneto,2694,346,3040,89650,92690,162,...,99108,4992,NaN,NaN,196790,3023689,1134294.0,45.0,NaN,NaN
5,4,2020-12-15T17:00:00,6,Friuli Venezia Giulia,654,56,710,13229,13939,-149,...,26082,1299,NaN,NaN,41320,828919,315194.0,3.0,NaN,NaN
6,5,2020-12-15T17:00:00,7,Liguria,768,71,839,7276,8115,-165,...,45234,2694,NaN,NaN,56043,655617,307046.0,3.0,NaN,NaN
7,6,2020-12-15T17:00:00,8,Emilia-Romagna,2797,214,3011,61623,64634,-352,...,76205,6719,NaN,NaN,147558,2349809,1157563.0,17.0,NaN,NaN
8,7,2020-12-15T17:00:00,9,Toscana,1156,214,1370,14894,16264,-1215,...,93619,3238,NaN,NaN,113121,1733863,1011122.0,15.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-15T17:00:00,10,Umbria,288,46,334,4226,4560,-175,...,21323,535,NaN,NaN,26418,460748,238473.0,7.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-16T17:00:00,1,Piemonte,3698,257,3955,44148,48103,-2867,1215,...,7302,NaN,NaN,188387,1801939,1054003.0,8.0,NaN,NaN,4341375
1,2020-12-16T17:00:00,2,Valle d'Aosta,67,7,74,449,523,-4,38,...,360,NaN,NaN,6943,59111,35825.0,1.0,NaN,NaN,125501
2,2020-12-16T17:00:00,3,Lombardia,4946,629,5575,64158,69733,-6157,2994,...,24097,NaN,NaN,450005,4514914,2489727.0,18.0,NaN,NaN,10103969
3,2020-12-16T17:00:00,4,Trentino-Alto Adige,617,72,689,12767,13456,119,529,...,1476,0.0,0.0,46315,757305,302339.0,4.0,0,NaN,1074819
4,2020-12-16T17:00:00,5,Veneto,2698,347,3045,91180,94225,1535,3817,...,5069,NaN,NaN,200607,3043792,1140117.0,31.0,NaN,NaN,4907704
5,2020-12-16T17:00:00,6,Friuli Venezia Giulia,634,54,688,13063,13751,-188,779,...,1339,NaN,NaN,42099,837745,317196.0,7.0,NaN,NaN,1211357
6,2020-12-16T17:00:00,7,Liguria,755,71,826,7203,8029,-86,275,...,2705,NaN,NaN,56318,660289,308177.0,5.0,Dei 120854 test antigenici rapidi sono riferit...,NaN,1543127
7,2020-12-16T17:00:00,8,Emilia-Romagna,2773,212,2985,59861,62846,-1788,1238,...,6797,NaN,NaN,148794,2367376,1162546.0,9.0,Il dato odierno cumulativo dei tamponi antigen...,NaN,4467118
8,2020-12-16T17:00:00,9,Toscana,1101,202,1303,13732,15035,-1229,489,...,3285,NaN,NaN,113610,1746053,1014930.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-16T17:00:00,10,Umbria,270,47,317,4006,4323,-237,169,...,541,NaN,NaN,26587,463909,239527.0,4.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-16T17:00:00,1,Piemonte,3698,257,3955,44148,48103,-2867,1215,...,NaN,NaN,188387,1801939,1054003.0,8.0,NaN,NaN,4341375,575
1,2020-12-16T17:00:00,2,Valle d'Aosta,67,7,74,449,523,-4,38,...,NaN,NaN,6943,59111,35825.0,1.0,NaN,NaN,125501,20
2,2020-12-16T17:00:00,3,Lombardia,4946,629,5575,64158,69733,-6157,2994,...,NaN,NaN,450005,4514914,2489727.0,18.0,NaN,NaN,10103969,1036
3,2020-12-16T17:00:00,4,Trentino-Alto Adige,617,72,689,12767,13456,119,529,...,0.0,0.0,46315,757305,302339.0,4.0,0,NaN,1074819,106
4,2020-12-16T17:00:00,5,Veneto,2698,347,3045,91180,94225,1535,3817,...,NaN,NaN,200607,3043792,1140117.0,31.0,NaN,NaN,4907704,1016
5,2020-12-16T17:00:00,6,Friuli Venezia Giulia,634,54,688,13063,13751,-188,779,...,NaN,NaN,42099,837745,317196.0,7.0,NaN,NaN,1211357,180
6,2020-12-16T17:00:00,7,Liguria,755,71,826,7203,8029,-86,275,...,NaN,NaN,56318,660289,308177.0,5.0,Dei 120854 test antigenici rapidi sono riferit...,NaN,1543127,209
7,2020-12-16T17:00:00,8,Emilia-Romagna,2773,212,2985,59861,62846,-1788,1238,...,NaN,NaN,148794,2367376,1162546.0,9.0,Il dato odierno cumulativo dei tamponi antigen...,NaN,4467118,634
8,2020-12-16T17:00:00,9,Toscana,1101,202,1303,13732,15035,-1229,489,...,NaN,NaN,113610,1746053,1014930.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-16T17:00:00,10,Umbria,270,47,317,4006,4323,-237,169,...,NaN,NaN,26587,463909,239527.0,4.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-16T17:00:00,5,Veneto,2698,347,3045,91180,94225,1535,3817,...,34.15,1530,497,20103,5823.0,77,2205,18.99,0.07778,4.09
1,2020-12-16T17:00:00,3,Lombardia,4946,629,5575,64158,69733,-6157,2994,...,60.71,-6080,590,37605,11576.0,106,9045,7.96,0.02963,4.45
2,2020-12-16T17:00:00,16,Puglia,1498,185,1683,51167,52850,575,1388,...,50.14,609,365,10188,4673.0,45,768,13.62,0.03463,1.90
3,2020-12-16T17:00:00,8,Emilia-Romagna,2773,212,2985,59861,62846,-1788,1238,...,33.44,-1762,0,17567,4983.0,78,2946,7.05,0.02771,3.33
4,2020-12-16T17:00:00,12,Lazio,2869,305,3174,78093,81267,-2954,1220,...,36.01,-2886,61,15705,7516.0,40,4134,7.77,0.02080,2.44
5,2020-12-16T17:00:00,1,Piemonte,3698,257,3955,44148,48103,-2867,1215,...,44.70,-2795,109,17493,7434.0,89,3993,6.95,0.02799,4.34
6,2020-12-16T17:00:00,19,Sicilia,1188,183,1371,33805,35176,-793,1065,...,31.12,-754,-22,9974,6444.0,29,1829,10.68,0.02144,1.64
7,2020-12-16T17:00:00,15,Campania,1701,124,1825,86379,88204,-2038,900,...,24.55,-2020,253,14757,6963.0,34,2904,6.10,0.01556,3.05
8,2020-12-16T17:00:00,6,Friuli Venezia Giulia,634,54,688,13063,13751,-188,779,...,30.00,-166,-50,8826,2002.0,40,927,8.83,0.06431,3.48
9,2020-12-16T17:00:00,4,Trentino-Alto Adige,617,72,689,12767,13456,119,529,...,67.92,127,64,5369,955.0,24,386,9.85,0.04922,4.31


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,72,34,106,67.92
1,Lombardia,629,407,1036,60.71
2,Marche,75,68,143,52.45
3,Puglia,185,184,369,50.14
4,Piemonte,257,318,575,44.70
5,Umbria,47,70,117,40.17
6,Toscana,202,321,523,38.62
7,Abruzzo,56,92,148,37.84
8,Lazio,305,542,847,36.01
9,Valle d'Aosta,7,13,20,35.00


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,2020-12-14T17:00:00,22,P.A. Trento,423,53,476,2078,2554,39,114,15134,781,NaN,NaN,18469,413133,144980.0,7.0,NaN,"Dei 114 casi odierni, 30 sono molecolari con p..."
590,2020-12-15T17:00:00,21,P.A. Bolzano,208,24,232,10398,10630,18,88,15653,657,NaN,NaN,26940,337081,155855.0,0.0,NaN,NaN
591,2020-12-15T17:00:00,22,P.A. Trento,412,53,465,2242,2707,153,377,15344,795,NaN,NaN,18846,414855,145529.0,3.0,NaN,"Dei 377 casi odierni, 279 sono molecolari con ..."
592,2020-12-16T17:00:00,21,P.A. Bolzano,203,22,225,10429,10654,24,285,15903,668,NaN,NaN,27225,339622,156374.0,1.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,2020-12-12T17:00:00,4,Trentino-Alto Adige,645,71,716,12342,13058,-32,472,...,16.0,5854.0,1384.0,-21.0,9.92,5.48,8.06,34.10,-4.05,-0.24
293,2020-12-13T17:00:00,4,Trentino-Alto Adige,634,74,708,12406,13114,56,408,...,6.0,4007.0,748.0,64.0,10.45,5.40,10.18,54.55,4.23,0.43
294,2020-12-14T17:00:00,4,Trentino-Alto Adige,644,79,723,12443,13166,52,182,...,25.0,1538.0,505.0,37.0,10.93,5.49,11.83,36.04,6.76,0.40
295,2020-12-15T17:00:00,4,Trentino-Alto Adige,620,77,697,12640,13337,171,465,...,22.0,2801.0,800.0,197.0,11.05,5.23,16.60,58.13,-2.53,1.30


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
